In [1]:
import pandas as pd
import matplotlib as plt
%matplotlib inline
import seaborn as sns
import pandas_profiling


In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [5]:
profile = train.profile_report(title='Haribon Profiling Report')
profile.to_file(output_file="haribon_pandas_profiling.html")

profile = test.profile_report(title='Test Haribon Profiling Report')
profile.to_file(output_file="test_haribon_pandas_profiling.html")

## Data Prep

In [7]:
# Test data for KNN

k_train = train
k_test = test
k_train = k_train.drop("indexId", axis=1)
k_test = k_test.drop("indexId", axis=1)

# Splitting for training

X_k_train = k_train.drop("Scientific_Name", axis=1)
Y_k_train = k_train['Scientific_Name']
X_k_train.head()

,Annual_Mean_Temperature,Mean_Diurnal_Range,Isothermality,Temperature_Seasonality,Max_Temperature_of_Warmest_Month,Min_Temperature_of_Coldest_Month,Temperature_Annual_Range,Mean_Temperature_of_Wettest_Quarter,Mean_Temperature_of_Driest_Quarter,Mean_Temperature_of_Warmest_Quarter,Mean_Temperature_of_Coldest_Quarter,Annual_Precipitation,Precipitation_of_Wettest_Month,Precipitation_of_Driest_Month,Precipitation_Seasonality,Precipitation_of_Wettest_Quarter,Precipitation_of_Driest_Quarter,Precipitation_of_Warmest_Quarter,Precipitation_of_Coldest_Quarter
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63.090,19.902,2432.0,25.639,117.493,420.0,7.148,25.357,462.0,178.0,896.0,11.330,26.975,52.334,31.232,53.0,305.0,26.028,23.993
3,66.983,20.849,2044.0,26.936,112.042,166.0,8.192,25.870,419.0,71.0,971.0,12.230,28.357,73.092,33.079,15.0,340.0,27.275,25.442
4,76.739,21.074,2113.0,26.943,63.116,569.0,9.091,27.528,589.0,429.0,663.0,11.847,27.757,26.627,32.921,135.0,298.0,27.095,26.139


### Feature engineering

In [9]:
import sys
from impyute.imputation.cs import fast_knn
sys.setrecursionlimit(100000)

### KNN Imputation

In [10]:
X_k_train = pd.DataFrame(fast_knn(X_k_train.values, k=30), columns = X_k_train.columns)

In [11]:
Y_k_train.unique()

array(['Cacatua haematuropygia', 'Anas luzonica', 'Pithecophaga jeffeyri',
       'Alcedo argentata', 'Egretta eulophotes'], dtype=object)

In [12]:
Y_k_train = Y_k_train.replace(['Cacatua haematuropygia', 'Anas luzonica', 'Pithecophaga jeffeyri',
       'Alcedo argentata', 'Egretta eulophotes'],[1,2,3,4,5])

## XGBOOST

In [13]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [19]:
XG = XGBClassifier(learning_rate = 0.04, n_estimators=300, max_depth=8)
XG.fit(X_k_train, Y_k_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.04, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

# Results

In [20]:
output = test
output['Predicted'] = XG.predict(k_test) 
output = output[['indexId', 'Predicted']]

In [21]:
output['Predicted'] = output['Predicted'].replace([1,2,3,4,5],['Cacatua haematuropygia', 'Anas luzonica', 'Pithecophaga jeffeyri',
       'Alcedo argentata', 'Egretta eulophotes'])

In [22]:
output.to_csv('cohortII_haribon_predict.csv',  index=False)

In [18]:
output['Predicted']

0     Cacatua haematuropygia
1         Egretta eulophotes
2     Cacatua haematuropygia
3     Cacatua haematuropygia
4      Pithecophaga jeffeyri
5              Anas luzonica
6      Pithecophaga jeffeyri
7      Pithecophaga jeffeyri
8           Alcedo argentata
9         Egretta eulophotes
10     Pithecophaga jeffeyri
11     Pithecophaga jeffeyri
12        Egretta eulophotes
13    Cacatua haematuropygia
14    Cacatua haematuropygia
15             Anas luzonica
16          Alcedo argentata
17     Pithecophaga jeffeyri
18          Alcedo argentata
19    Cacatua haematuropygia
20    Cacatua haematuropygia
21    Cacatua haematuropygia
22     Pithecophaga jeffeyri
23    Cacatua haematuropygia
24    Cacatua haematuropygia
25    Cacatua haematuropygia
26             Anas luzonica
27             Anas luzonica
28     Pithecophaga jeffeyri
29        Egretta eulophotes
               ...          
50             Anas luzonica
51    Cacatua haematuropygia
52     Pithecophaga jeffeyri
53     Pitheco